# Sharing Processes with a Team

## Connection and Cursor

We will use a connection object to define our connection to our database over a network.

We will use a cursor object to write to the database. The cursor is created by a class function of the connection object.

In [1]:
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor

connection = pg2.connect(host='this_postgis',
                         user='postgres',
                         database='postgres')
cursor = connection.cursor(cursor_factory=RealDictCursor)

The cursor can be used to execute queries and then to fetch the results of the query.

In [2]:
result = cursor.execute("SELECT * FROM business LIMIT 2;")

In [3]:
cursor.fetchall()

[{'id': 64660,
  'name': 'HAIGHT STREET MARKET',
  'address': '1530 HAIGHT ST ',
  'city': 'San Francisco',
  'state': 'CA',
  'postal_code': 94117,
  'latitude': 37.769957,
  'longitude': -122.447533,
  'phone_number': '+14152550643',
  'gpnt_location': '0101000020E6100000C32973F38DE242405FB87361A49C5EC0'},
 {'id': 5632,
  'name': 'AT&T - DOGGIE DINNER - RM 5333/V [145162]',
  'address': '24 WILLIE MAYS PLAZA 5.26.01  ',
  'city': 'San Francisco',
  'state': 'CA',
  'postal_code': 94107,
  'latitude': 37.778382,
  'longitude': -122.391151,
  'phone_number': '+14159721500',
  'gpnt_location': '0101000020E6100000C5E57805A2E342400D33349E08995EC0'}]

In [4]:
connection.close()

Because we used the `cursor_factory=RealDictCursor` argument, the results return a list of dictionary objects from the database. 

The advantage of this is that we can display these results easily with a DataFrame.


In [5]:
import pandas as pd

In [6]:
connection = pg2.connect(host='this_postgis',
                         user='postgres',
                         database='postgres')
cursor = connection.cursor(cursor_factory=RealDictCursor)
result = cursor.execute("SELECT * FROM business LIMIT 2;")
pd.DataFrame(cursor.fetchall())

,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
0,1530 HAIGHT ST,San Francisco,0101000020E6100000C32973F38DE242405FB87361A49C...,64660,37.769957,-122.447533,HAIGHT STREET MARKET,+14152550643,94117,CA
1,24 WILLIE MAYS PLAZA 5.26.01,San Francisco,0101000020E6100000C5E57805A2E342400D33349E0899...,5632,37.778382,-122.391151,AT&T - DOGGIE DINNER - RM 5333/V [145162],+14159721500,94107,CA


In [7]:
connection.close()

## `lib.db_helper`

Because we will keep using the connection-cursor pattern, we have written it into a sub-module.

In [8]:
cd /home/jovyan

/home/jovyan


In [9]:
import lib.db_helper as db

#### `db.connect_to_db()`

Contains our connection credentials and returns a connection and cursor. 

    def connect_to_db():
        con = pg2.connect(host='postgis',
                          dbname='postgres',
                          user='postgres')
        cur = con.cursor(cursor_factory=RealDictCursor)
        return con, cur

In [10]:
connection, cursor = db.connect_to_db()
result = cursor.execute("SELECT * FROM business LIMIT 2;")
pd.DataFrame(cursor.fetchall())

,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
0,1530 HAIGHT ST,San Francisco,0101000020E6100000C32973F38DE242405FB87361A49C...,64660,37.769957,-122.447533,HAIGHT STREET MARKET,+14152550643,94117,CA
1,24 WILLIE MAYS PLAZA 5.26.01,San Francisco,0101000020E6100000C5E57805A2E342400D33349E0899...,5632,37.778382,-122.391151,AT&T - DOGGIE DINNER - RM 5333/V [145162],+14159721500,94107,CA


In [11]:
connection.close()

#### `db.query_to_dictionary()`

1. creates a connection and a cursor
1. uses the cursor to execute a query
1. if `fetch_res` is `True` it fetches the results, otherwise results are `None`
1. closes the connection and returns `results`


    def query_to_dictionary(query, fetch_res=True):
        con, cur = connect_to_db()
        cur.execute(query)
        if fetch_res:
            results = cur.fetchall()
        else:
            results = None
        con.close()
        return results

In [12]:
db.query_to_dictionary("""SELECT * FROM business LIMIT 2;""")

[{'id': 64660,
  'name': 'HAIGHT STREET MARKET',
  'address': '1530 HAIGHT ST ',
  'city': 'San Francisco',
  'state': 'CA',
  'postal_code': 94117,
  'latitude': 37.769957,
  'longitude': -122.447533,
  'phone_number': '+14152550643',
  'gpnt_location': '0101000020E6100000C32973F38DE242405FB87361A49C5EC0'},
 {'id': 5632,
  'name': 'AT&T - DOGGIE DINNER - RM 5333/V [145162]',
  'address': '24 WILLIE MAYS PLAZA 5.26.01  ',
  'city': 'San Francisco',
  'state': 'CA',
  'postal_code': 94107,
  'latitude': 37.778382,
  'longitude': -122.391151,
  'phone_number': '+14159721500',
  'gpnt_location': '0101000020E6100000C5E57805A2E342400D33349E08995EC0'}]

In [13]:
db.query_to_dictionary("""SELECT * FROM business LIMIT 2;""", fetch_res=False)

#### `db.query_to_dataframe()`

Wraps `query_to_dictionary()` in a `pandas.DataFrame`.

    def query_to_dataframe(query):
        return DataFrame(query_to_dictionary(query))

In [14]:
db.query_to_dataframe('SELECT * FROM business LIMIT 2;')

,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
0,1530 HAIGHT ST,San Francisco,0101000020E6100000C32973F38DE242405FB87361A49C...,64660,37.769957,-122.447533,HAIGHT STREET MARKET,+14152550643,94117,CA
1,24 WILLIE MAYS PLAZA 5.26.01,San Francisco,0101000020E6100000C5E57805A2E342400D33349E0899...,5632,37.778382,-122.391151,AT&T - DOGGIE DINNER - RM 5333/V [145162],+14159721500,94107,CA


### Create new column in `business` table

To demonstrate how transactions work, we will write a SQL query to create a new column in our database.

#### Run this if the column has already been created and you wish to run all cells in the notebook

In [15]:
# db.query_to_dictionary("""
# BEGIN;
# ALTER TABLE business DROP COLUMN gpnt_location;
# COMMIT;
# """, fetch_res=False)

In [16]:
db.query_to_dictionary("""
BEGIN;
ALTER TABLE business ADD COLUMN gpnt_location geometry(POINT,4326);
COMMIT;
""", fetch_res=False)

ProgrammingError: column "gpnt_location" of relation "business" already exists


In [17]:
db.query_to_dataframe('SELECT * FROM business LIMIT 2;')

,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
0,1530 HAIGHT ST,San Francisco,0101000020E6100000C32973F38DE242405FB87361A49C...,64660,37.769957,-122.447533,HAIGHT STREET MARKET,+14152550643,94117,CA
1,24 WILLIE MAYS PLAZA 5.26.01,San Francisco,0101000020E6100000C5E57805A2E342400D33349E0899...,5632,37.778382,-122.391151,AT&T - DOGGIE DINNER - RM 5333/V [145162],+14159721500,94107,CA


# Clean the Data

In [18]:
select_count_business = """
SELECT COUNT(*) FROM business
"""
db.query_to_dataframe(select_count_business)

,count
0,6138


In [19]:
select_count_by_postal_code = """
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
"""
db.query_to_dataframe(select_count_by_postal_code)

,count,postal_code
0,1,0.0
1,2,94013.0
2,1,94101.0
3,447,94102.0
4,552,94103.0
5,132,94104.0
6,225,94105.0
7,429,94107.0
8,223,94108.0
9,373,94109.0


Note that we might also have invalid data.

In [20]:
select_count_business_invalid_data = """
SELECT COUNT(*) FROM business
WHERE latitude = 0 AND longitude = 0;"""

In [21]:
db.query_to_dataframe(select_count_business_invalid_data)

,count
0,14


In [22]:
select_count_business_valid = """
SELECT COUNT(*) FROM business
WHERE 
    (latitude IS NOT NULL
     AND longitude IS NOT NULL)
AND 
    (latitude != 0 
     AND longitude != 0)
"""

select_count_business_invalid = """
SELECT COUNT(*) FROM business
WHERE 
    (latitude IS NULL
     OR longitude IS NULL)
OR 
    (latitude = 0 
     OR longitude = 0)
"""

In [23]:
db.query_to_dataframe(select_count_business_valid)

,count
0,3735


In [24]:
db.query_to_dataframe(select_count_business_invalid)

,count
0,2403


### Store Valid and Invalid Queries

We will also make use of subqueries. We will define two:

1. a subquery for all businesses with valid lat/long
1. a subquery for all businesses with invalid lat/long

#### Valid Business Query

In [25]:
select_business_valid = """
SELECT * FROM business
WHERE 
    (latitude IS NOT NULL
     AND longitude IS NOT NULL)
AND 
    (latitude != 0 
     AND longitude != 0)
"""

select_business_invalid = """
SELECT * FROM business
WHERE 
    (latitude IS NULL
     OR longitude IS NULL)
OR 
    (latitude = 0 
     OR longitude = 0)
"""

In [26]:
print(db.query_to_dataframe(select_business_valid).shape)
db.query_to_dataframe(select_business_valid).sample(4)

(3735, 10)


,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
3516,570 LARKIN ST,San Francisco,0101000020E61000009BCB0D863AE44240CC5EB69DB69A...,68774,37.783036,-122.417396,FOX MARKET,+14154410690,94102.0,CA
491,4124 GEARY BLVD,San Francisco,0101000020E6100000FCAA5CA8FCE342408A0453CDAC9D...,1903,37.781148,-122.463672,GEARY STREET BELLA PIZZA,+14156684150,94118.0,CA
1094,737 DIAMOND ST,San Francisco,0101000020E6100000E38E37F92DE04240F86C1D1CEC9B...,3867,37.751403,-122.436286,BACCO,+14152824969,94114.0,CA
149,2401 NORIEGA ST,San Francisco,0101000020E6100000DB17D00B77E042405E85949F549F...,510,37.753633,-122.489540,DONUT TIME,+14157538997,94122.0,CA


In [27]:
print(db.query_to_dataframe(select_business_invalid).shape)
db.query_to_dataframe(select_business_invalid).sample(4)

(2403, 10)


,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
539,226 KEARNY ST,San Francisco,None,73849,NaN,NaN,SUSHIRRITO,+14153229549,94108.0,CA
1312,1788 32ND AVE,San Francisco,None,79626,NaN,NaN,BUBBLE CAFE,+14158302940,94122.0,CA
2094,2001 CHESTNUT ST,San Francisco,None,84619,NaN,NaN,THE DORIAN,+14152180203,94123.0,CA
511,5501 CALIFORNIA ST,San Francisco,None,72002,NaN,NaN,JAPONICA,,94121.0,CA


### Explore Postal Code and Lat/Long

First, let's get the postal codes with a count of less than 10. We will use a sub-query.

Let's use these subqueries to help us to clean the data. 

Note that we re-use `select_count_by_postal_code`.

In [28]:
print(select_count_by_postal_code)


SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code



In [29]:
select_postal_code_by_postal_code_less_than_10 = """
SELECT postal_code FROM 
    ({}) this_subquery
WHERE count < 10
""".format(select_count_by_postal_code)

print(select_postal_code_by_postal_code_less_than_10)


SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) this_subquery
WHERE count < 10



Note that we are using a subquery here. 

e.g. 

    SELECT * FROM (SUBQUERY) sub_query_name;
    

In [30]:
db.query_to_dataframe(select_postal_code_by_postal_code_less_than_10)

,postal_code
0,0
1,94013
2,94101
3,94120
4,94130
5,94140
6,94143
7,94513
8,94609
9,95105


We'll think of these as our "bad postal codes". We can think of the query we defined as a list of these codes. That query was

In [31]:
bad_postal_codes = select_postal_code_by_postal_code_less_than_10

In [32]:
print(bad_postal_codes)


SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) this_subquery
WHERE count < 10



Do we have (lat,long) pairs for these locations? Let's select all rows with one of these postal codes.

### `IN`

To do this we will use the SQL keyword `IN`. `IN` checks a column against a list of items.

e.g. 

    SELECT * FROM my_table WHERE my_column IN ('1,2,3,4,5');

# We will replace the list with our Sub-Query!! 

In [33]:
select_business_where_bad_postal_code = """
SELECT * FROM business WHERE postal_code IN ({})""".format(bad_postal_codes)
print(select_business_where_bad_postal_code)


SELECT * FROM business WHERE postal_code IN (
SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) this_subquery
WHERE count < 10
)


In [34]:
db.query_to_dataframe(select_business_where_bad_postal_code).sample(5)

,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
11,41 MONTGOMERY ST,San Francisco,0101000020E6100000923B6C2233E34240C53C2B69C59A...,65856,37.774998,-122.418299,BAMBOO ASIA,+14156246790,94101,CA
9,500 PARNASSUS AVE J LEVEL,San Francisco,0101000020E61000008330B77BB9E142408E3D7B2E539D...,65425,37.763473,-122.458202,PANDA EXPRESS,+14156657957,94143,CA
5,"2 AVE OF PALMS BLDG 146, TREASURE ISLAND",San Francisco,0101000020E6100000C6DFF60489E94240F7ADD689CB97...,38139,37.824494,-122.371798,ISLAND MARKET & DELI,+14158600303,94130,CA
10,PUBLIC RIGHT OF WAY,San Francisco,0101000020E6100000B7291E17D5EA42404E637B2DE890...,65673,37.834628,-122.264171,DOC'S OF THE BAY,+14156883245,94609,CA
0,820 BRYANT STREET,San Francisco,0101000020E61000009D2D20B41EF0424069519FE40E6D...,3827,37.875937,-121.704034,MCDONALDS,+14158645282,94513,CA


Note that some of these have a lat and long. 

What we want is rows that have "bad postal codes" and no (lat, long) pair. 

### A Nested Query

To get these we will write a nested query that looks like this:

    SELECT * FROM (invalid_lat_long) WHERE postal_code IN (bad_postal_code_str)

In [35]:
select_invalid_business_where_bad_postal_code = """
SELECT * 
FROM (""" + select_business_invalid + """) this_subquery
WHERE postal_code IN (""" + bad_postal_codes + """)
"""
print(select_invalid_business_where_bad_postal_code)


SELECT * 
FROM (
SELECT * FROM business
WHERE 
    (latitude IS NULL
     OR longitude IS NULL)
OR 
    (latitude = 0 
     OR longitude = 0)
) this_subquery
WHERE postal_code IN (
SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) this_subquery
WHERE count < 10
)



In [36]:
select_invalid_business_where_bad_postal_code = """
SELECT * 
FROM ({}) q
WHERE postal_code IN ({})
""".format(select_business_invalid, bad_postal_codes)
print(select_invalid_business_where_bad_postal_code)


SELECT * 
FROM (
SELECT * FROM business
WHERE 
    (latitude IS NULL
     OR longitude IS NULL)
OR 
    (latitude = 0 
     OR longitude = 0)
) q
WHERE postal_code IN (
SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) this_subquery
WHERE count < 10
)



In [37]:
db.query_to_dataframe(select_invalid_business_where_bad_postal_code)

""


#### Let's use `DELETE` to get rid of rows that have no lat, long and a bad zip

First, let's collect a list of business `id`s. 

In [38]:
select_invalid_business_id_where_bad_postal_code = """
SELECT id 
FROM ({}) q
WHERE postal_code IN ({})
""".format(select_business_invalid, select_postal_code_by_postal_code_less_than_10)
print(select_invalid_business_id_where_bad_postal_code)


SELECT id 
FROM (
SELECT * FROM business
WHERE 
    (latitude IS NULL
     OR longitude IS NULL)
OR 
    (latitude = 0 
     OR longitude = 0)
) q
WHERE postal_code IN (
SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) this_subquery
WHERE count < 10
)



In [39]:
db.query_to_dataframe(select_invalid_business_id_where_bad_postal_code)

""


In [40]:
print(select_invalid_business_id_where_bad_postal_code)


SELECT id 
FROM (
SELECT * FROM business
WHERE 
    (latitude IS NULL
     OR longitude IS NULL)
OR 
    (latitude = 0 
     OR longitude = 0)
) q
WHERE postal_code IN (
SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) this_subquery
WHERE count < 10
)



We can use this subquery in our `DELETE` query.

Remember, that a **delete** action is a write action and needs to be handled as a transaction. We need to `BEGIN` and `COMMIT`. 

In [41]:
delete_invalid_business_bad_postal_code = """
BEGIN;
DELETE 
FROM business
WHERE id IN ({});
COMMIT;
""".format(select_invalid_business_id_where_bad_postal_code)
print(delete_invalid_business_bad_postal_code)


BEGIN;
DELETE 
FROM business
WHERE id IN (
SELECT id 
FROM (
SELECT * FROM business
WHERE 
    (latitude IS NULL
     OR longitude IS NULL)
OR 
    (latitude = 0 
     OR longitude = 0)
) q
WHERE postal_code IN (
SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) this_subquery
WHERE count < 10
)
);
COMMIT;



Remember, when we execute this we do not need to fetch results so we should set `fetch_res` to `False`.

In [42]:
db.query_to_dictionary(delete_invalid_business_bad_postal_code, fetch_res=False)

Let's have a look at the sql file used to define our database. 

```SQL
CREATE TABLE business (
    id INTEGER,
    name TEXT,
    address TEXT,
    city TEXT,
    state TEXT,
    postal_code INTEGER,
    latitude FLOAT,
    longitude FLOAT,
    phone_number TEXT,
    PRIMARY KEY (id));

CREATE TABLE inspection (
    business_id INTEGER,
    score INTEGER,
    date TIMESTAMP,
    type TEXT,
    CONSTRAINT fk_business_id
    FOREIGN KEY (business_id)
    REFERENCES business (id));

CREATE TABLE legend (
    minimum_score INTEGER,
    maximum_score INTEGER,
    description TEXT);

CREATE TABLE violation (
    business_id INTEGER,
    date TIMESTAMP,
    description TEXT,
    CONSTRAINT fk_business_id
    FOREIGN KEY (business_id)
    REFERENCES business (id));

```

Note that we have foreign key `CONSTRAINT`s on the `inspection` and `violation` tables.  

This means that, in order to drop the businesses with "bad" data, we will need to drop any inspections and violations associated with these. 

### "Bad" Business Data

In [43]:
db.query_to_dataframe(select_invalid_business_id_where_bad_postal_code)

""


In [44]:
print(select_invalid_business_id_where_bad_postal_code)


SELECT id 
FROM (
SELECT * FROM business
WHERE 
    (latitude IS NULL
     OR longitude IS NULL)
OR 
    (latitude = 0 
     OR longitude = 0)
) q
WHERE postal_code IN (
SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) this_subquery
WHERE count < 10
)



In [45]:
select_violations_for_bad_biz = """
SELECT * FROM violation
WHERE business_id IN ({})
""".format(select_invalid_business_id_where_bad_postal_code)

In [46]:
db.query_to_dataframe(select_violations_for_bad_biz)

""


### Delete Violations

In [47]:
select_count_violation = """
SELECT COUNT(*) FROM violation"""

In [48]:
db.query_to_dataframe(select_count_violation)

,count
0,39863


In [49]:
delete_violations_for_bad_biz = """
BEGIN;
DELETE FROM violation
WHERE business_id IN ({});
COMMIT;""".format(select_invalid_business_id_where_bad_postal_code)

In [50]:
db.query_to_dictionary(delete_violations_for_bad_biz, fetch_res=False)

In [51]:
db.query_to_dataframe(select_count_violation)

,count
0,39863


### Delete Inspections

In [52]:
select_count_inspections = """
SELECT COUNT(*) FROM inspection"""

In [53]:
db.query_to_dataframe(select_count_inspections)

,count
0,15408


In [54]:
delete_inspections_for_bad_biz = """
BEGIN;
DELETE FROM inspection
WHERE business_id IN ({});
COMMIT;""".format(select_invalid_business_id_where_bad_postal_code)

In [ ]:
db.query_to_dictionary(delete_inspections_for_bad_biz, fetch_res=False)

In [ ]:
db.query_to_dataframe(select_count_inspections)

### Delete Businesses

In [ ]:
db.query_to_dataframe(select_count_business)

In [ ]:
db.query_to_dictionary(delete_invalid_business_bad_postal_code, fetch_res=False)

In [ ]:
db.query_to_dataframe(select_count_business)

### Pull Bad Postal Codes Again

In [ ]:
db.query_to_dataframe(select_postal_code_by_postal_code_less_than_10)

## Repair Bad Zip Codes
To repair the bad zip codes, we will attempt match the nearest points using GIS. 

To do this, we will first need to populate the `gpnt_location` column.

In [ ]:
db.query_to_dataframe(select_business_valid).sample(4)

In [ ]:
update_gpnt = """
BEGIN;
UPDATE business SET 
gpnt_location = ST_SetSRID(ST_MakePoint(latitude, longitude),4326);
COMMIT;
"""

In [ ]:
db.query_to_dictionary(update_gpnt, fetch_res=False)

### Match points to the points with bad zip codes using a given radius

In [ ]:
db.query_to_dataframe(select_postal_code_by_postal_code_less_than_10)

In [ ]:
def select_postal_code(postal_code):
    return """SELECT * FROM business WHERE postal_code = {}""".format(postal_code)

In [ ]:
db.query_to_dataframe(select_postal_code(92672))

In [64]:
def select_gpnt_for_postal_code(postal_code):
    return """SELECT gpnt_location FROM business WHERE postal_code = {} LIMIT 1""".format(postal_code)

In [62]:
def select_on_gpnt_radius(postal_code, distance):
    return """
            SELECT *
            FROM business WHERE ST_Distance_Sphere(gpnt_location, ({})) <= {}
            """.format(select_gpnt_for_postal_code(postal_code), distance)

In [ ]:
db.query_to_dataframe(select_on_gpnt_radius(92672, 100))

In [ ]:
update_92672 = """
BEGIN;
UPDATE business SET postal_code = 94117 WHERE postal_code = 92672;
COMMIT;
"""

In [ ]:
db.query_to_dictionary(update_92672, fetch_res=False)

### Check the Status of our Bad Postal Codes

In [ ]:
db.query_to_dataframe(select_postal_code_by_postal_code_less_than_10)

In [ ]:
db.query_to_dataframe(select_on_gpnt_radius(94101, 100))

## Plot locations

Next, we will use Folium to plot the businesses for which we have a latitude and a longitude. 

### `folium.Map` 

To build our `folium.Map` object we will need to center the map in a given location. We will use the `avg` aggregate function to do this. 

In [55]:
import folium

In [56]:
avg_location_sf = """
SELECT avg(latitude) as avg_lat, 
       avg(longitude) as avg_lon FROM ({}) q;
""".format(select_business_valid)

In [57]:
db.query_to_dataframe(avg_location_sf)

,avg_lat,avg_lon
0,37.774003,-122.425144


In [58]:
avg_loc_df = db.query_to_dataframe(avg_location_sf)

Then assign the result as a single list value.

In [59]:
avg_loc = avg_loc_df.values.tolist()[0]

In [60]:
sf_map = folium.Map(location=avg_loc, zoom_start=13)

In [65]:
within_200_meters_of_94101 = db.query_to_dataframe(select_on_gpnt_radius(94101, 100))

In [66]:
near_94101_lat_long = within_200_meters_of_94101[['latitude', 'longitude','postal_code']].values.tolist()

In [69]:
for loc in near_94101_lat_long:
    try:
        postal_code = str(int(loc[2]))
    except:
        postal_code = None
    folium.Marker(loc[:2],postal_code).add_to(sf_map)
print(near_94101_lat_long)

[[37.773467, -122.418093, 94103.0], [37.775975, -122.418129, 94103.0], [37.775977, -122.418126, 94103.0], [37.773447, -122.418123, 94103.0], [37.774998, -122.418299, 94101.0], [37.776154, -122.418049, 94102.0], [37.776144, -122.418058, 94102.0]]


In [68]:
sf_map